# 构建您自己的文本分类模型
## — 利用数字资产盘活机器人解决方案训练您自己的模型

借助 Amazon SageMaker，您可以打包自己的算法，然后进行训练和部署，以便与数字资产盘活机器人解决方案框架一起使用。本笔记本将引导您完成使用 TensorFlow bert预训练模型，训练一个文本分类模型，用于评论数据的舆情监控，本示例向您展示如何为 SageMaker 构建 Docker 容器并将其用于培训和推理。


## 权限

运行此笔记本电脑除了正常的 “SageMakerFullAccess” 权限之外，还需要权限。这是因为它在 Amazon ECR 中创建了新的存储库。添加这些权限的最简单方法是将托管策略 “AmazonEC2ContainerRegistryFullAccess” 添加到您用于启动笔记本实例的角色中。执行此操作时无需重新启动笔记本实例，新权限将立即可用。

## 示例

在此示例中，我们演示了如何使用 Python 示例打包自定义 TensorFlow 容器，该示例与注释数据集一起工作，并使用 TensorFlow 服务进行推理。但是，可以通过修改码头容器使用 TensorFlow 服务以外的不同推理解决方案。

在此示例中，我们使用单个映像来支持培训和托管。这简化了过程，因为我们只需要为两个任务管理一个映像。有时，您可能需要单独的图像进行培训和托管，因为它们有不同的要求。在这种情况下，将下面讨论的部分分开到单独的 Dockerfiles 中并构建两个映像。选择是使用单个映像还是使用两个映像是开发和管理最方便的问题。

如果您只使用 Amazon SageMaker 进行培训或托管，但不是同时使用这两种功能，则只需要将所使用的功能内置到您的容器中。

## 数据

本例子中的训练数据使用的是包含 9600 个标记为 0/1 的中文评论
* 0 代表负面评论
* 1 代表积极的评论。

用于测试的 1200 个/用于开发的 1200 个

In [9]:
import pandas as pd
import sagemaker

# 第一部分：数据探索

In [10]:
#preview
df_train = pd.read_csv('../data/train.tsv',sep='\t')
df_train.head()

,label,text_a
0,1,选择珠江花园的原因就是方便，有电动扶梯直接到达海边，周围餐馆、食廊、商场、超市、摊位一应俱全...
1,1,15.4寸笔记本的键盘确实爽，基本跟台式机差不多了，蛮喜欢数字小键盘，输数字特方便，样子也很...
2,0,房间太小。其他的都一般。。。。。。。。。
3,0,"1.接电源没有几分钟,电源适配器热的不行. 2.摄像头用不起来. 3.机盖的钢琴漆，手不能摸..."
4,1,"今天才知道这书还有第6卷,真有点郁闷:为什么同一套书有两种版本呢?当当网是不是该跟出版社商量..."


In [1]:
from sagemaker import get_execution_role

role = get_execution_role()

## 拟合、部署、预测

现在，我们使用 “train” 调用我们的 TensorFlow 容器，并将我们的超参数和其他元数据作为容器内的 /opt/ml/输入/配置中的json 文件传递。

训练成功后，我们的训练算法将训练模型输出到 /opt/ml/模型目录中，该目录用于处理预测。

然后，我们可以使用实例计数和实例类型调用 `部署（）`，即 1 和 `本地 '。这使用 “服务” 调用我们的 Tensorflow 容器，它设置我们的容器来处理通过 TensorFlow 服务的预测请求。返回的是一个预测变量，它用于根据我们训练的模型进行推断。

在我们的预测之后，我们可以删除我们的终端节点。

我们建议您首先在本地测试和训练您的训练算法，因为它提供了更快的迭代和更好的可调试性。

# 第 二 部分：在SageMaker 中培训和托管您的算法
一旦您的容器打包，您就可以使用它来培训和服务模型。

## 设置环境
在这里，我们指定要使用的存储桶和用于使用 SageMaker 的角色。

In [2]:
# S3 prefix
prefix = 'bert-switch'

In [3]:
import sagemaker as sage

sess = sage.Session()

## 上传数据至s3用于训练

We will use the tools provided by the SageMaker Python SDK to upload the data to a default bucket.

In [4]:
# first download data from a open source website 
WORK_DIRECTORY = '../data'

data_location = sess.upload_data(WORK_DIRECTORY, key_prefix=prefix)

In [5]:
data_location

's3://sagemaker-cn-northwest-1-251885400447/bert-switch'

## 使用 SageMaker 进行模型训练
使用 Python SDK 在 SageMaker 上训练模型的方式与我们在本地训练模型的方式类似。这是通过将我们的实例类型从 “本地” 更改为我们的 [支持的 EC2 实例类型] (https://aws.amazon.com/sagemaker/pricing/instance-types/) 来完成的。

此外，我们现在必须指定 ECR 镜像 URL，是我们提前本地构建推送的

最后，我们的本地培训数据集必须位于 Amazon S3 中，并且数据集的 S3 URL 传递到 `fit () `调用中。

让我们首先获取与我们刚刚建立和推送的镜像对应的 ECR 镜像网址

In [12]:
ecr_image = '251885400447.dkr.ecr.cn-northwest-1.amazonaws.com.cn/bert-switch'

In [ ]:
from sagemaker.estimator import Estimator


instance_type = 'ml.p3.2xlarge'


hyperparameters = {'task_name':'chnsenticorp',
                   'vocab_file':'./pretrain_model/chinese_L-12_H-768_A-12/vocab.txt',
                   'bert_config_file':'./pretrain_model/chinese_L-12_H-768_A-12/bert_config.json',
                   'output_dir':'/opt/ml/model',
                   'export_model_dir':'/opt/ml/model/export/Servo',
                   'data_dir':'/opt/ml/input/data/training',
                   'do_train':'true',
                   'do_predict':'true',
                   'do_eval':'true',
                   'init_checkpoint':'./pretrain_model/chinese_L-12_H-768_A-12/bert_model.ckpt',
                   'max_seq_length':128,
                   'num_train_epochs':2.0}

estimator = Estimator(role=role,
                      train_instance_count=1,
                      train_instance_type=instance_type,
                      image_name=ecr_image,
                      hyperparameters=hyperparameters,
                      train_volume_size=512)
estimator.fit(data_location)

Parameter image_name will be renamed to image_uri in SageMaker Python SDK v2.
's3_input' class will be renamed to 'TrainingInput' in SageMaker Python SDK v2.


2020-09-01 10:37:57 Starting - Starting the training job...
2020-09-01 10:38:00 Starting - Launching requested ML instances......
2020-09-01 10:39:24 Starting - Preparing the instances for training.........
2020-09-01 10:40:43 Downloading - Downloading input data...

In [15]:
estimator

## 模型部署

In [17]:
predictor = estimator.deploy(1, instance_type='ml.m5.large', endpoint_name='bert-sentiment')

Parameter image will be renamed to image_uri in SageMaker Python SDK v2.


---------------!

## 本地测试

In [18]:
import sys
sys.path.append("./bert")

import tokenization
from extract_features import InputExample, convert_examples_to_features
import numpy as np
import requests
import os
import time

vocab_file = os.environ.get('vocab_file', './bert/pretrain_model/chinese_L-12_H-768_A-12/vocab.txt')
max_token_len = os.environ.get('max_token_len', 128)


def preprocess(text):
    text_a = text
    example = InputExample(unique_id=None, text_a=text_a, text_b=None)
    tokenizer = tokenization.FullTokenizer(
        vocab_file=vocab_file, do_lower_case=True)
    feature = convert_examples_to_features([example], max_token_len, tokenizer)[0]
    input_ids = np.reshape([feature.input_ids], (1, max_token_len))
    return {
        "inputs": {"input_ids": input_ids.tolist()}
    }

In [19]:
from sagemaker.predictor import json_serializer, json_deserializer

text = "测试一下啦！"
data = preprocess(text)

predictor.accept = 'application/json'
predictor.content_type = 'application/json'

predictor.serializer = json_serializer
predictor.deserializer = json_deserializer

start = time.time()
resp = predictor.predict(data)
end = time.time()
pro_0, pro_1 = resp['outputs'][0]
print(f"negative pro:{pro_0} positive pro:{pro_1} time consuming:{int((end - start) * 1000)}ms")





INFO:tensorflow:*** Example ***
INFO:tensorflow:unique_id: None
INFO:tensorflow:tokens: [CLS] 测 试 一 下 啦 ！ [SEP]
INFO:tensorflow:input_ids: 101 3844 6407 671 678 1568 8013 102 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
INFO:tensorflow:input_mask: 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
INFO:tensorflow:input_type_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
negative pr

## 资源清理

当你完成端点时，你应该清理它。

您可以通过 AWS 账户的 SageMaker 控制台查看我们创建的所有培训作业、模型和终端节点。


In [ ]:
predictor.delete_endpoint()

# Reference
- [How Amazon SageMaker interacts with your Docker container for training](https://docs.aws.amazon.com/sagemaker/latest/dg/your-algorithms-training-algo.html)
- [How Amazon SageMaker interacts with your Docker container for inference](https://docs.aws.amazon.com/sagemaker/latest/dg/your-algorithms-inference-code.html)
- [SageMaker Python SDK](https://github.com/aws/sagemaker-python-sdk)
- [Dockerfile](https://docs.docker.com/engine/reference/builder/)
- [scikit-bring-your-own](https://github.com/awslabs/amazon-sagemaker-examples/blob/master/advanced_functionality/scikit_bring_your_own/scikit_bring_your_own.ipynb)